In [1]:
batch_size = 64
model_img_size = 299 #TODO: 224
tta = True
best_thr = 0.085

In [2]:
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import math
import PIL
from PIL import ImageOps
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, applications
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import backend as K 
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras

from tqdm.auto import tqdm
tqdm.pandas()

Using TensorFlow backend.


In [3]:
train_path = "../input/imet-2019-fgvc6/train/"
test_path = "../input/imet-2019-fgvc6/test/"

In [4]:
df_labels = pd.read_csv("../input/imet-2019-fgvc6/labels.csv")
df_labels.head()

,attribute_id,attribute_name
0,0,culture::abruzzi
1,1,culture::achaemenid
2,2,culture::aegean
3,3,culture::afghan
4,4,culture::after british


In [5]:
n_classes = df_labels.shape[0]
n_classes

1103

In [6]:
from imgaug import augmenters as iaa

def img_augment(img):
    #fifty_chance = lambda aug: iaa.Sometimes(0.5, aug)
    import random
    if random.randint(1,101) > 90:
        #print("original")
        #one in 10 return original image
        return img
    seq = iaa.SomeOf(3, [
        iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
        iaa.Fliplr(0.5), # horizontally flip 50% of the images
        #iaa.GaussianBlur(sigma=(0, 1.0)), # blur images with a sigma of 0 to 1.0 TODO: test for good values
        iaa.Affine(
            #scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            scale=(0.8, 1.2),
            cval=0, # if mode is constant, use a cval between 0 and 255
            mode="constant" # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        ),
        iaa.Affine(
            translate_percent={"x": (-0.2, 0.2), "y": 0}, # translate by -20 to +20 percent (per axis)
            cval=0, # if mode is constant, use a cval between 0 and 255
            mode="constant" # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        ),
        iaa.Affine(
            rotate=(-10, 10), # rotate by -10 to +10 degrees
            #order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=0, # if mode is constant, use a cval between 0 and 255
            mode="constant" # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        ),
        iaa.Affine(
            shear=(-5, 5), # shear by -5 to +5 degrees
            #order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=0, # if mode is constant, use a cval between 0 and 255
            mode="constant" # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        ),
    ])
    img = seq.augment_image(img)
    return img

#https://www.kaggle.com/mathormad/resnet50-v2-keras-focal-loss-mix-up
#https://github.com/aleju/imgaug

In [7]:
from keras.applications import resnet50
from keras.applications import inception_resnet_v2
from keras.applications import inception_v3
import random

def img_pad_resize(filename, input_path, augment, preprocessor):
    img = PIL.Image.open(f'{input_path}/{filename}')
    img_w = img.size[0]
    img_h = img.size[1]

    cropped = False
    if random.randint(1,101) > 50:
        aspect_ratio = img_w/img_h
        if aspect_ratio > 2 or aspect_ratio < 0.5:
            diff = abs(img_w-img_h)
            if img_w > img_h:
                crop_size = img_h
                crop_y = 0
                crop_x = random.randint(0, diff)
            else:
                #print(filename)
                crop_size = img_w
                crop_x = 0
                crop_y = random.randint(0, diff)
            img_cropped = img.crop((crop_x, crop_y, crop_x+crop_size, crop_y+crop_size))
            img.close()
            img = img_cropped
            cropped = True
            #print("cropped:"+filename)
       
    w = img.size[0]
    h = img.size[1]
    pad_size = np.abs(h-w)
    wm = hm = 1
    pw = ph = 0
    if w < h:
        wm = h / w
        pw = pad_size / 2
    else:
        hm = w / h
        ph = pad_size / 2
    w *= wm
    h *= hm
    h = int(h)
    w = int(w)
    pw = int(pw)
    ph = int(ph)
    padding = (pw, ph, pw, ph)
    padded = ImageOps.expand(img, padding)
    resized = padded.resize((model_img_size, model_img_size))
    np_img = np.array(resized)
    img.close()
    padded.close()
    del img
    del padded
    if augment:
        np_img = img_augment(np_img)
    np_img = preprocessor(np_img)
    #np_img = resnet50.preprocess_input(np_img)
    return np_img

In [8]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a.iloc[p], b[p]

In [9]:
from skimage.io import imread
from skimage.transform import resize
from keras.utils import Sequence

#https://github.com/sdcubber/Keras-Sequence-boilerplate/blob/master/Keras-Sequence.ipynb
# Here, `x_set` is list of path to the images
# and `y_set` are the associated classes.

class MySequence(Sequence):

    def __init__(self, x_set, y_set, batch_size, mode="train", augment=True):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.mode = mode
        self.max_idx = math.ceil(len(x_set)/batch_size)
        self.augment = augment

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx2 = idx % self.max_idx
        #i guess the len method above stops the generator from being called too much
        #print(f"idx:{idx}, idx2:{idx2}")
        start = idx * self.batch_size
        end = min(start + batch_size, len(self.x))
        batch_x = self.x.iloc[start : end]
        batch_y = self.y[start : end]
        
        next_batch = []
        for index, row in batch_x.iterrows():
            file_name = row["filename"]
            file_path = row["path"]
            padded = img_pad_resize(file_name, file_path, self.augment)
            if padded.shape != (299, 299, 3):
                print(f"shape mismatch {file_name}, {file_path}, {padded.shape}")
                print()
                #the image has alpha channel, drop it
                padded = padded[ :, :, :3]
            
            next_batch.append(padded)
        np_y = np.array(batch_y)
        result = np.array(next_batch), np_y

        #print(result[0].shape)
        return result

    def on_epoch_end(self):
        self.x, self.y = unison_shuffled_copies(self.x, self.y)

In [10]:
# create callbacks list
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)
                             
checkpoint = ModelCheckpoint('../working/Resnet50_best_{epoch:03d}_{val_loss:.2f}.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3,
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=7)

csv_logger = CSVLogger(filename='../working/training_log.csv',
                       separator=',',
                       append=True)

callbacks_list = [checkpoint, csv_logger, early, reduceLROnPlat]

/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [11]:
def plot_gen_batch(idx):
    # configure batch size and retrieve one batch of images
    plt.clf() #clears matplotlib data and axes
    #for batch in train_generator:
    rows = batch_size / 3
    plt.figure(figsize=[30,10*rows])
    batch = train_gen.__getitem__(idx)
    for x in range(0,batch_size-1):
    #    print(train_generator.filenames[x])
        plt.subplot(rows, 3, x+1)
        plt.imshow(batch[0][x], interpolation='nearest')

        item_labels = np.argwhere(batch[1][x] > 0)
        title_val = []
        info_str = str(item_labels)
        for tag_id in item_labels:
            att_name = df_labels[df_labels['attribute_id'] == tag_id[0]]
            info_str += " "+str(att_name["attribute_name"]) #att_name is a dataframe
            title_val.append(att_name['attribute_name'].iloc[0]) #i
        #print(info_str)
        plt.title(title_val)

    plt.show()

In [12]:
from keras_applications import imagenet_utils as utils

def ResNet(stack_fn,
           preact,
           use_bias,
           model_name='resnet',
           include_top=True,
           weights='imagenet',
           input_tensor=None,
           input_shape=None,
           pooling=None,
           classes=1000,
           **kwargs):
    """Instantiates the ResNet, ResNetV2, and ResNeXt architecture.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        stack_fn: a function that returns output tensor for the
            stacked residual blocks.
        preact: whether to use pre-activation or not
            (True for ResNetV2, False for ResNet and ResNeXt).
        use_bias: whether to use biases for convolutional layers or not
            (True for ResNet and ResNetV2, False for ResNeXt).
        model_name: string, model name.
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels.
        pooling: optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    global backend, layers, models, keras_utils
    # backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)
    backend, layers, models, keras_utils = keras.backend, keras.layers, keras.models, keras.utils

    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = utils._obtain_input_shape(input_shape,
                                          default_size=224,
                                          min_size=32,
                                          data_format=backend.image_data_format(),
                                          require_flatten=include_top,
                                          weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, 7, strides=2, use_bias=use_bias, name='conv1_conv')(x)

    if preact is False:
        x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                      name='conv1_bn')(x)
        x = layers.Activation('relu', name='conv1_relu')(x)

    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name='pool1_pad')(x)
    x = layers.MaxPooling2D(3, strides=2, name='pool1_pool')(x)

    x = stack_fn(x)

    if preact is True:
        x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                      name='post_bn')(x)
        x = layers.Activation('relu', name='post_relu')(x)

    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        x = layers.Dense(classes, activation='softmax', name='probs')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D(name='max_pool')(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model.
    model = models.Model(inputs, x, name=model_name)

    # Load weights.
    if (weights == 'imagenet') and (model_name in WEIGHTS_HASHES):
        if include_top:
            file_name = model_name + '_weights_tf_dim_ordering_tf_kernels.h5'
            file_hash = WEIGHTS_HASHES[model_name][0]
        else:
            file_name = model_name + '_weights_tf_dim_ordering_tf_kernels_notop.h5'
            file_hash = WEIGHTS_HASHES[model_name][1]
        weights_path = keras_utils.get_file(file_name,
                                            BASE_WEIGHTS_PATH + file_name,
                                            cache_subdir='models',
                                            file_hash=file_hash)
        model.load_weights(weights_path)
    elif weights is not None:
        model.load_weights(weights)

    return model

In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

def block2(x, filters, kernel_size=3, stride=1,
           conv_shortcut=False, name=None):
    """A residual block.
    # Arguments
        x: input tensor.
        filters: integer, filters of the bottleneck layer.
        kernel_size: default 3, kernel size of the bottleneck layer.
        stride: default 1, stride of the first layer.
        conv_shortcut: default False, use convolution shortcut if True,
            otherwise identity shortcut.
        name: string, block label.
    # Returns
        Output tensor for the residual block.
    """
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    preact = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                       name=name + '_preact_bn')(x)
    preact = layers.Activation('relu', name=name + '_preact_relu')(preact)

    if conv_shortcut is True:
        shortcut = layers.Conv2D(4 * filters, 1, strides=stride,
                                 name=name + '_0_conv')(preact)
    else:
        shortcut = layers.MaxPooling2D(1, strides=stride)(x) if stride > 1 else x

    x = layers.Conv2D(filters, 1, strides=1, use_bias=False,
                      name=name + '_1_conv')(preact)
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_1_bn')(x)
    x = layers.Activation('relu', name=name + '_1_relu')(x)

    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name + '_2_pad')(x)
    x = layers.Conv2D(filters, kernel_size, strides=stride,
                      use_bias=False, name=name + '_2_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_2_bn')(x)
    x = layers.Activation('relu', name=name + '_2_relu')(x)

    x = layers.Conv2D(4 * filters, 1, name=name + '_3_conv')(x)
    x = layers.Add(name=name + '_out')([shortcut, x])
    return x


def stack2(x, filters, blocks, stride1=2, name=None):
    """A set of stacked residual blocks.
    # Arguments
        x: input tensor.
        filters: integer, filters of the bottleneck layer in a block.
        blocks: integer, blocks in the stacked blocks.
        stride1: default 2, stride of the first layer in the first block.
        name: string, stack label.
    # Returns
        Output tensor for the stacked blocks.
    """
    x = block2(x, filters, conv_shortcut=True, name=name + '_block1')
    for i in range(2, blocks):
        x = block2(x, filters, name=name + '_block' + str(i))
    x = block2(x, filters, stride=stride1, name=name + '_block' + str(blocks))
    return x

def ResNet50V2(include_top=True,
               weights='imagenet',
               input_tensor=None,
               input_shape=None,
               pooling=None,
               classes=1000,
               **kwargs):
    def stack_fn(x):
        x = stack2(x, 64, 3, name='conv2')
        x = stack2(x, 128, 4, name='conv3')
        x = stack2(x, 256, 6, name='conv4')
        x = stack2(x, 512, 3, stride1=1, name='conv5')
        return x
    return ResNet(stack_fn, True, True, 'resnet50v2',
                  include_top, weights,
                  input_tensor, input_shape,
                  pooling, classes,
                  **kwargs)


In [14]:
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)
from keras.applications.resnet50 import ResNet50

def create_model_resnet(trainable_layer_count):
    input_tensor = Input(shape=(model_img_size, model_img_size, 3))
    base_model = ResNet50V2(include_top=False,
                   weights=None,
                   input_tensor=input_tensor)
    base_model.load_weights('../input/keras-pretrain-model-weights/resnet50v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
#    base_model = ResNet50(include_top=False,
#                          #the weights value can apparently also be a file path..
#                   weights=None, #loading weights from dataset, avoiding need for internet conn
#                   input_tensor=input_tensor)
#    base_model.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    if trainable_layer_count == "all":
        for layer in base_model.layers:
            layer.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False
        for layer in base_model.layers[-trainable_layer_count:]:
            layer.trainable = True
    print("base model has {} layers".format(len(base_model.layers)))
#     x = Conv2D(32, kernel_size=(1,1), activation='relu')(base_model.output)
#     x = Flatten()(x)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(5e-4))(x)
    x = Dropout(0.5)(x)
    #predict individual probability of each category
    final_output = Dense(n_classes, activation='sigmoid', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [15]:
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2

def create_model_inception_resnet(trainable_layer_count):
    input_tensor = Input(shape=(model_img_size, model_img_size, 3))
    base_model = InceptionResNetV2(include_top=False,
                   weights=None,
                   input_tensor=input_tensor)
    base_model.load_weights('../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
#    base_model = ResNet50(include_top=False,
#                          #the weights value can apparently also be a file path..
#                   weights=None, #loading weights from dataset, avoiding need for internet conn
#                   input_tensor=input_tensor)
#    base_model.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    if trainable_layer_count == "all":
        for layer in base_model.layers:
            layer.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False
        for layer in base_model.layers[-trainable_layer_count:]:
            layer.trainable = True
    print("base model has {} layers".format(len(base_model.layers)))
#     x = Conv2D(32, kernel_size=(1,1), activation='relu')(base_model.output)
#     x = Flatten()(x)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(5e-4))(x)
    x = Dropout(0.5)(x)
    #predict individual probability of each category
    final_output = Dense(n_classes, activation='sigmoid', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [16]:
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

def create_model_inception(trainable_layer_count):
    input_tensor = Input(shape=(model_img_size, model_img_size, 3))
    base_model = InceptionV3(include_top=False,
                   weights=None,
                   input_tensor=input_tensor)
    base_model.load_weights('../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
#    base_model = ResNet50(include_top=False,
#                          #the weights value can apparently also be a file path..
#                   weights=None, #loading weights from dataset, avoiding need for internet conn
#                   input_tensor=input_tensor)
#    base_model.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    if trainable_layer_count == "all":
        for layer in base_model.layers:
            layer.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False
        for layer in base_model.layers[-trainable_layer_count:]:
            layer.trainable = True
    print("base model has {} layers".format(len(base_model.layers)))
#     x = Conv2D(32, kernel_size=(1,1), activation='relu')(base_model.output)
#     x = Flatten()(x)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(5e-4))(x)
    x = Dropout(0.5)(x)
    #predict individual probability of each category
    final_output = Dense(n_classes, activation='sigmoid', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [17]:
gamma = 2.0
epsilon = K.epsilon()
def focal_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = K.pow(1-pt, gamma) * CE
    loss = K.sum(FL, axis=1)
    return loss

In [18]:
def f2_score(y_true, y_pred):
    print(f"y_true="+str(y_true[0]))
    print(f"y_pred={y_pred}")
    beta = 2
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=1)
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    result = K.mean(((1+beta**2)*precision*recall) / ((beta**2)*precision+recall+K.epsilon()))
    print(result)
    return result



In [19]:
beta_f2=2

def my_f2(y_true, y_pred):
    print(f"y_true="+str(y_true[0]))
    print(f"y_pred={y_pred}")
#    assert y_true.shape[0] == y_pred.shape[0]

    tp = np.sum((y_true == 1) & (y_pred == 1), axis=1)
    tn = np.sum((y_true == 0) & (y_pred == 0), axis=1)
    fp = np.sum((y_true == 0) & (y_pred == 1), axis=1)
    fn = np.sum((y_true == 1) & (y_pred == 0), axis=1)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f2 = (1+beta_f2**2)*p*r / (p*beta_f2**2 + r + 1e-15)

    return np.mean(f2)

In [20]:
from sklearn.metrics import fbeta_score

#https://www.kaggle.com/anokas/fixed-f2-score-in-python
def my_my_f2(y_true, y_pred):
    # fbeta_score throws a confusing error if inputs are not numpy arrays
    y_true, y_pred, = np.array(y_true), np.array(y_pred)
    # We need to use average='samples' here, any other average method will generate bogus results
    return fbeta_score(y_true, y_pred, beta=2, average='samples')


In [21]:
from keras.optimizers import Adam
#f2_score = my_f2
#https://datascience.stackexchange.com/questions/26112/decay-parameter-in-keras-optimizers
model_resnet = create_model_resnet("all")
model_resnet.compile(optimizer=Adam(lr=0.0001, decay=0.0001), loss='binary_crossentropy', metrics=["accuracy", f2_score]) #loss=focal_loss
model_resnet.load_weights("../input/imetresnet50v2trainedweights/Resnet50_best_013_0.01.h5")

Instructions for updating:
Colocations handled automatically by placer.
base model has 190 layers
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
y_true=Tensor("metrics/f2_score/strided_slice:0", shape=(?,), dtype=float32)
y_pred=Tensor("final_output/Sigmoid:0", shape=(?, 1103), dtype=float32)
Tensor("metrics/f2_score/Mean:0", shape=(), dtype=float32)


In [22]:
model_inception_resnet = create_model_inception_resnet("all")
model_inception_resnet.compile(optimizer=Adam(lr=0.0001, decay=0.0001), loss='binary_crossentropy', metrics=["accuracy", f2_score]) #loss=focal_loss
model_inception_resnet.load_weights("../input/imetresnet50v2trainedweights/inception_resnet_best_004_0.01.h5")

base model has 780 layers
y_true=Tensor("metrics_1/f2_score/strided_slice:0", shape=(?,), dtype=float32)
y_pred=Tensor("final_output_1/Sigmoid:0", shape=(?, 1103), dtype=float32)
Tensor("metrics_1/f2_score/Mean:0", shape=(), dtype=float32)


In [23]:
model_inception = create_model_inception("all")
model_inception.compile(optimizer=Adam(lr=0.0001, decay=0.0001), loss='binary_crossentropy', metrics=["accuracy", f2_score]) #loss=focal_loss
model_inception.load_weights("../input/imetresnet50v2trainedweights/Inception_best_008_0.01.h5")

base model has 311 layers
y_true=Tensor("metrics_2/f2_score/strided_slice:0", shape=(?,), dtype=float32)
y_pred=Tensor("final_output_2/Sigmoid:0", shape=(?, 1103), dtype=float32)
Tensor("metrics_2/f2_score/Mean:0", shape=(), dtype=float32)


In [24]:
%%time

probabilities_resnet = []
submit_resnet = pd.read_csv('../input/imet-2019-fgvc6/sample_submission.csv')

for name in tqdm(submit_resnet['id']):
    np_img = img_pad_resize(name+".png", test_path, False, resnet50.preprocess_input)
    if tta:
        img_pred = np.zeros(n_classes)
        for x in range(10):
            aug_img = img_augment(np_img)
            score_predict = model_resnet.predict(aug_img[np.newaxis])
            img_pred += np.squeeze(score_predict)
        img_pred /= 10
        score_predict = img_pred
    else:
        score_predict = model_resnet.predict(np_img[np.newaxis])
    probabilities_resnet.append(score_predict)


In [25]:
%%time

probabilities_inception_resnet = []
submit_inception_resnet = pd.read_csv('../input/imet-2019-fgvc6/sample_submission.csv')

for name in tqdm(submit_inception_resnet['id']):
    np_img = img_pad_resize(name+".png", test_path, False, inception_resnet_v2.preprocess_input)
    if tta:
        img_pred = np.zeros(n_classes)
        for x in range(6):
            aug_img = img_augment(np_img)
            score_predict = model_inception_resnet.predict(aug_img[np.newaxis])
            img_pred += np.squeeze(score_predict)
        img_pred /= 6
        score_predict = img_pred
    else:
        score_predict = model_inception_resnet.predict(np_img[np.newaxis])
    probabilities_inception_resnet.append(score_predict)


In [26]:
%%time

probabilities_inception = []
submit_inception = pd.read_csv('../input/imet-2019-fgvc6/sample_submission.csv')

for name in tqdm(submit_inception['id']):
    np_img = img_pad_resize(name+".png", test_path, False, inception_v3.preprocess_input)
    if tta:
        img_pred = np.zeros(n_classes)
        for x in range(8):
            aug_img = img_augment(np_img)
            score_predict = model_inception.predict(aug_img[np.newaxis])
            img_pred += np.squeeze(score_predict)
        img_pred /= 8
        score_predict = img_pred
    else:
        score_predict = model_inception.predict(np_img[np.newaxis])
    probabilities_inception.append(score_predict)

In [27]:
np_pred = np.array(probabilities)

NameError: name 'probabilities' is not defined

In [28]:
np.set_printoptions(suppress=True)
np_pred[0:2]

NameError: name 'np_pred' is not defined

In [29]:
probabilities_inception[0:2]

[array([0.00002833, 0.00006745, 0.00033819, ..., 0.00003007, 0.00015867,
        0.00007873]),
 array([0.00000085, 0.00006013, 0.00025669, ..., 0.00001119, 0.0000319 ,
        0.00003241])]

In [30]:
np_prob_inception = np.array(probabilities_inception)
np_prob_inception_resnet = np.array(probabilities_inception_resnet)
np_prob_resnet = np.array(probabilities_resnet)


In [31]:
np_prob_sum = np_prob_inception + np_prob_inception_resnet + np_prob_resnet
#np_prob_sum = np_prob_inception + np_prob_resnet
np_prob_sum[0:2]

array([[0.00010678, 0.00051504, 0.00125169, ..., 0.00017352, 0.00107907,
        0.00028112],
       [0.00029389, 0.00278867, 0.0004789 , ..., 0.00003994, 0.00078088,
        0.00017206]])

In [32]:
np_prob_avg = np_prob_sum/3
np_prob_avg[0:2]

array([[0.00003559, 0.00017168, 0.00041723, ..., 0.00005784, 0.00035969,
        0.00009371],
       [0.00009796, 0.00092956, 0.00015963, ..., 0.00001331, 0.00026029,
        0.00005735]])

In [33]:
#np_pred_inception.shape

In [34]:
label_predict = np_prob_avg>=best_thr
label_predict = label_predict.astype(int)
label_predict[0:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
#np.argwhere(label_predict[0] == 1).flatten()

In [36]:
#numeroiksi nuo binary ennusteet
indices = []
for row in label_predict:
    new_row = np.argwhere(row == 1).flatten()
    str_predict_label = ' '.join(str(l) for l in new_row)
    indices.append(str_predict_label)
indices[0:3]

['369 586 587 766 1039 1059', '188 231 369 1039', '79 121 415 498 961']

In [37]:
#majority vote
label_predict_inception = np_prob_inception>=best_thr
label_predict_inception = label_predict_inception.astype(int)

label_predict_inception_resnet = np_prob_inception_resnet>=best_thr
label_predict_inception_resnet = label_predict_inception_resnet.astype(int)

label_predict_resnet = np_prob_resnet>=best_thr
label_predict_resnet = label_predict_resnet.astype(int)

label_pred_sum = label_predict_inception + label_predict_inception_resnet + label_predict_resnet
#label_pred_sum = label_predict_inception + label_predict_resnet
label_pred_sum[0:2]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
#numeroiksi nuo binary ennusteet
indices_major = []
for row in label_pred_sum:
    new_row = np.argwhere(row >= 3).flatten()
    str_predict_label = ' '.join(str(l) for l in new_row)
    indices_major.append(str_predict_label)
indices_major[0:3]

['369', '', '79']

In [39]:
#np.argwhere(label_pred_sum[0] == 1).flatten()

In [40]:
#np.argwhere(label_pred_sum[0] >= 2).flatten()

In [41]:
submit = pd.read_csv('../input/imet-2019-fgvc6/sample_submission.csv')
submit['attribute_ids'] = indices
submit.to_csv('submission.csv', index=False)

In [42]:
submit.head()

,id,attribute_ids
0,10023b2cc4ed5f68,369 586 587 766 1039 1059
1,100fbe75ed8fd887,188 231 369 1039
2,101b627524a04f19,79 121 415 498 961
3,10234480c41284c6,13 51 111 147 480 483 737 738 776 813 830 923 ...
4,1023b0e2636dcea8,51 147 156 227 322 477 489 584 612 671 738 813...


In [43]:
submit = pd.read_csv('../input/imet-2019-fgvc6/sample_submission.csv')
submit['attribute_ids'] = indices_major
submit.to_csv('submission.csv', index=False)

In [44]:
submit.head()

,id,attribute_ids
0,10023b2cc4ed5f68,369
1,100fbe75ed8fd887,
2,101b627524a04f19,79
3,10234480c41284c6,13 147 480 483 738 776 830 1046
4,1023b0e2636dcea8,147 322 584 612 738 813 954 1046 1092
